In [3]:
import pandas as pd
import numpy as np

fp = "/home/gathomp3/deep_learning/EIG_elicitation/GATE-forked/model_model_results/website_preferences/gpt-4_at_end_0_all_test_results.csv"
df = pd.read_csv(fp)
df

,Unnamed: 0,interaction_time,interaction_num_turns,interaction_total_char_length,accuracy,AUCROC,correct_prob,accuracy_relative,AUCROC_relative,correct_prob_relative,question_mode,task,engine,seed
0,0,NaN,"[0, 1]",NaN,"[0.5, 0.6875]","[0.5546875, 0.75]","[0.5375, 0.6000000000000001]","[0.0, 0.1875]","[0.0, 0.1953125]","[0.0, 0.06250000000000011]",questions_open,website_preferences,gpt-4,0
1,1,NaN,"[0, 1]",NaN,"[0.5625, 0.8125]","[0.6454545454545455, 0.9909090909090909]","[0.5625, 0.7500000000000001]","[0.0, 0.25]","[0.0, 0.34545454545454535]","[0.0, 0.1875000000000001]",questions_open,website_preferences,gpt-4,0
2,2,NaN,"[0, 1]",NaN,"[0.5, 0.625]","[0.5, 0.890625]","[0.5125000000000001, 0.64375]","[0.0, 0.125]","[0.0, 0.390625]","[0.0, 0.13124999999999998]",questions_open,website_preferences,gpt-4,0
3,3,NaN,"[0, 1]",NaN,"[0.625, 0.875]","[0.6416666666666666, 1.0]","[0.5875, 0.7812500000000001]","[0.0, 0.25]","[0.0, 0.3583333333333334]","[0.0, 0.1937500000000001]",questions_open,website_preferences,gpt-4,0
4,4,NaN,"[0, 1]",NaN,"[0.5, 0.75]","[0.5166666666666667, 0.8750000000000001]","[0.55, 0.70625]","[0.0, 0.25]","[0.0, 0.3583333333333334]","[0.0, 0.15625]",questions_open,website_preferences,gpt-4,0
